### Convert a list of nodes defining a route to a set of coordinates

#### Make sure that OSM API is imported and can be invoked

In [ ]:
import osmapi

In [ ]:
osm = osmapi.OsmApi()

In [ ]:
osm.NodeGet(272208711)

#### Convert a list of route waypoints to lat lng

In [ ]:
route_ids = \
[
272208711, 189968414, 65544615,
65434053, 5935985475, 1782988537, 1782988370,
473404902, 6426708498, 343270779, 4177165168,
4183935315, 343603930, 4177165169, 343270132,
290554459, 65466965, 222327728, 5262671552,
65587153, 65587151, 65628960, 65628958, 65628956,
65470892, 147451990, 1615001111, 1735909450
]

In [ ]:
def node_to_geojson_coords(node_id):
    node_details = osm.NodeGet(node_id)
    return [node_details["lon"], node_details["lat"]]

In [ ]:
route_coords = [node_to_geojson_coords(node_id) for node_id in route_ids]

In [ ]:
route_coords

#### Option 1: Use OSRM to find a route

In [ ]:
route_coords_string = ";".join([",".join([str(lon), str(lat)]) for [lon, lat] in route_coords])

In [ ]:
route_coords_string

In [ ]:
import requests

##### With the same options as OSM (figured out by looking at the console logs) -> geometries are in the individual legs

In [ ]:
basic_url = "https://routing.openstreetmap.de/routed-car/route/v1/driving/"+route_coords_string+"?overview=false&geometries=polyline&steps=true"

In [ ]:
basic_result = requests.get(basic_url); basic_result

In [ ]:
basic_result_json = basic_result.json(); basic_result_json

In [ ]:
len(basic_result_json["routes"])

In [ ]:
basic_result_json["routes"][0]

In [ ]:
basic_result_json["waypoints"]

In [ ]:
basic_result_json["routes"][0]

##### Settings steps = false, we have no geometries at all

In [ ]:
basic_url = "https://routing.openstreetmap.de/routed-car/route/v1/driving/"+route_coords_string+"?overview=false&geometries=polyline&steps=false"
basic_result = requests.get(basic_url); basic_result
basic_result_json = basic_result.json(); basic_result_json
basic_result_json["routes"][0]

##### Setting overview = full returns an overall geometry

In [ ]:
##### Now we have no geometries
basic_url = "https://routing.openstreetmap.de/routed-car/route/v1/driving/"+route_coords_string+"?overview=full&geometries=polyline&steps=true"
basic_result = requests.get(basic_url); basic_result
basic_result_json = basic_result.json(); basic_result_json
basic_result_json

### Plotting the waypoints

##### Waypoints - seems to map to points that we passed in

In [ ]:
import folium

In [ ]:
def lonlat_swap(lon_lat):
    return list(reversed(lon_lat))

In [ ]:
basic_result_map = folium.Map()
for i, w in enumerate(basic_result_json["waypoints"]):
    basic_result_map.add_child(folium.Marker(lonlat_swap(w["location"]), popup="%d: %s" % (i, w["name"])))
basic_result_map.fit_bounds([lonlat_swap(c) for c in route_coords])
basic_result_map

##### Decoding the geometry (note that we have one geometry per leg)

In [ ]:
import polyline.codec as pc

In [ ]:
print("About to decode "+basic_result_json["routes"][0]["geometry"])

In [ ]:
decoded_geometry = pc.PolylineCodec().decode(basic_result_json["routes"][0]["geometry"]); decoded_geometry

##### Comparing waypoints to geometry, there are a LOT more geometry points

In [ ]:
len(basic_result_json["waypoints"]), len(decoded_geometry)

In [ ]:
basic_geometry_map = folium.Map()
folium.PolyLine(decoded_geometry).add_to(basic_geometry_map)
basic_geometry_map.fit_bounds([c for c in decoded_geometry])
basic_geometry_map

##### Too much zig-zagging, probably due to too many waypoints, reducing them

In [ ]:
route_ids = [272208711, 343270779, 343270132,
            65587153, 147451990, 1735909450
]

In [ ]:
route_coords = [node_to_geojson_coords(node_id) for node_id in route_ids]
route_coords_string = ";".join([",".join([str(lon), str(lat)]) for [lon, lat] in route_coords])
basic_url = "https://routing.openstreetmap.de/routed-car/route/v1/driving/"+route_coords_string+"?overview=full&geometries=polyline&steps=true"
basic_result = requests.get(basic_url); basic_result
basic_result_json = basic_result.json(); basic_result_json
basic_result_json

##### Redoing with car

In [ ]:
basic_result_map = folium.Map()
for i, w in enumerate(basic_result_json["waypoints"]):
    basic_result_map.add_child(folium.Marker(lonlat_swap(w["location"]), popup="%d: %s" % (i, w["name"])))
basic_result_map.fit_bounds([lonlat_swap(c) for c in route_coords])
basic_result_map

In [ ]:
decoded_geometry = pc.PolylineCodec().decode(basic_result_json["routes"][0]["geometry"]); decoded_geometry
basic_geometry_map = folium.Map()
folium.PolyLine(decoded_geometry).add_to(basic_geometry_map)
basic_geometry_map.fit_bounds([c for c in decoded_geometry])
basic_geometry_map

##### This makes a lot more sense, but let's try it with foot mode

In [ ]:
route_coords = [node_to_geojson_coords(node_id) for node_id in route_ids]
route_coords_string = ";".join([",".join([str(lon), str(lat)]) for [lon, lat] in route_coords])
basic_url = "https://routing.openstreetmap.de/routed-foot/route/v1/driving/"+route_coords_string+"?overview=full&geometries=polyline&steps=true"
basic_result = requests.get(basic_url); print(basic_result)
basic_result_json = basic_result.json(); basic_result_json
basic_result_json

In [ ]:
basic_result_map = folium.Map()
for i, w in enumerate(basic_result_json["waypoints"]):
    basic_result_map.add_child(folium.Marker(lonlat_swap(w["location"]), popup="%d: %s" % (i, w["name"])))
basic_result_map.fit_bounds([lonlat_swap(c) for c in route_coords])
basic_result_map

In [ ]:
decoded_geometry = pc.PolylineCodec().decode(basic_result_json["routes"][0]["geometry"]); decoded_geometry
basic_geometry_map = folium.Map()
folium.PolyLine(decoded_geometry).add_to(basic_geometry_map)
for i, c in enumerate(decoded_geometry):
    folium.CircleMarker(c, radius=5, popup="%d: %s" % (i, c)).add_to(basic_geometry_map)
basic_geometry_map.fit_bounds([c for c in decoded_geometry])
basic_geometry_map